In [0]:
from sklearn.datasets import load_iris

In [0]:
x = load_iris().data
y = load_iris().target

In [0]:
from sklearn.model_selection import train_test_split

In [4]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
y = to_categorical(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name = 'input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, y.shape[1]], name='input_labels')
  
  with tf.name_scope('model'):
    weights = tf.Variable(tf.random_normal(shape=[x.shape[1], y.shape[1]]), name = 'weights')
    biases = tf.Variable(tf.random_normal(shape=[y.shape[1]]))
    linear_model = tf.add(tf.matmul(input_features, weights), biases)
    predictions = tf.nn.softmax(linear_model)
    
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=linear_model, labels=input_labels))
  train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)
  
with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions,1),tf.argmax(input_labels,1)),tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)
  
summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='./logs', graph=tf.get_default_graph())

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0,stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [0]:
LOG_DIR = './logs'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [13]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.6/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [14]:
with tf.Session() as sess:
  sess.run(init_op)
  
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0]/16)):
      batch_x, batch_y = next_batch(batch_size=16, features=x_train, labels=y_train)
      
      _, train_loss, train_accuracy = sess.run([train_op, loss, accuracy_op, summary],feed_dict={input_features:batch_x, input_labels:batch_y})
      writer.add_summary(summary=train_summary, global_step=mini_batch)
    print('Epoch {}, loss: {}, accuracy: {}'.format(epoch, train_loss,train_accuracy))

ValueError: ignored